In [1]:
import transformers
from transformers import Trainer, BitsAndBytesConfig
import torch

2024-01-10 00:23:41.630957: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/var/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:106: UserWarning: 

BNB_CUDA_VERSION=XXX can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64
Loading CUDA version: BNB_CUDA_VERSION=122


  warn((f'\n\n{"="*80}\n'


[2024-01-10 00:23:43,461] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
model_name_or_path = "../../../../../model/vicuna-7b-v1.3"
load_in_4bit = True

In [3]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)
model = transformers.AutoModelForCausalLM.from_pretrained(
        model_name_or_path,
        low_cpu_mem_usage=True,
        torch_dtype=torch.bfloat16,
        quantization_config=quantization_config if load_in_4bit else None,
        load_in_4bit=load_in_4bit,
    )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
model_max_length=1024

In [8]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
        model_name_or_path,
        model_max_length=model_max_length,
        padding_side="right",
        use_fast=False,
    )
tokenizer.pad_token = tokenizer.unk_token

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [11]:
import json
filename = "../../../../../data/ShareGPT_Vicuna_unfiltered/small_test.json"
with open(filename, 'r') as f:
    train = json.load(f)

In [134]:
Q = {'question':[]}

In [135]:
for i in train:
    for j  in i['conversations']:
        if j['from'] =='human':
            print(j)
            Q['question'].append(j)
            break

{'from': 'human', 'value': 'What is the goal of dating?'}
{'from': 'human', 'value': 'and add condition styling if time more than "9:00"'}
{'from': 'human', 'value': 'Can you write me a homily on a particular subject?'}
{'from': 'human', 'value': 'أكتبلي سكريبت اعلان لشركة هواتف محمولة لهاتف للفئة المتوسطة موجه للشباب مدمنى العاب الفيديو جيمز'}
{'from': 'human', 'value': 'let\'s take Andy Clark\'s view here from his book "natural born cyborgs"'}
{'from': 'human', 'value': 'I want to design an event that signify the concept of The Force, and create a fun, light-hearted, positive, imaginative feeling. Which movie/event is the best to based on then?'}
{'from': 'human', 'value': 'Explain in detail "What are the key factors that determine the success or failure of organizational change initiatives?"'}
{'from': 'human', 'value': 'Steps to configure Azure server to upload a site that includes Front and Back end'}
{'from': 'human', 'value': "ok so i need to plan for tomorrow. i'll give you my 

In [136]:
with open( "../../../../../data/ShareGPT_Vicuna_unfiltered/small_question.json", "w") as json_file:
    json.dump(Q, json_file)

In [137]:
with open( "../../../../../data/ShareGPT_Vicuna_unfiltered/small_question.json", "r") as json_file:
   Qs =  json.load( json_file)

In [96]:
filename =  "../../../../../data/ShareGPT_Vicuna_unfiltered/train_question.json"

In [131]:
for i in range(len(Q['question'])):
    if i % 500 == 0:
        with open( filename, "w") as json_file:
            json.dump(Q,json_file)
        print("已经保存{}个question".format(i))
    input = tokenizer([Q['question'][i]['value']])
    output = model.generate(inputs = torch.tensor(input['input_ids'] ),max_length = len(input['input_ids'][0])*8,early_stopping =True)
    text = tokenizer.decode(output[0],skip_special_tokens =True)
    ques['question'][i]['answer'] = text

已经保存0个question



KeyboardInterrupt

